# Climate and Soil

This script outputs the satellite-based rasters into the Google Earth Engine Cloud.

Inputs:


Outputs:




In [12]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder

## Soil Data from SoilGrids

250m resolution

In [14]:
bdod = ee.Image("projects/soilgrids-isric/bdod_mean").clip(roi) # bulk density
cec = ee.Image("projects/soilgrids-isric/cec_mean") # cation exchange capacity
clay = ee.Image("projects/soilgrids-isric/clay_mean").clip(roi)
nitro = ee.Image("projects/soilgrids-isric/nitrogen_mean").clip(roi)
ocd = ee.Image("projects/soilgrids-isric/ocd_mean").clip(roi) # organic carbon density
ocs = ee.Image("projects/soilgrids-isric/ocs_mean").clip(roi) # soil organic carbon stock
phh2o = ee.Image("projects/soilgrids-isric/phh2o_mean").clip(roi)
sand = ee.Image("projects/soilgrids-isric/sand_mean").clip(roi)
soc = ee.Image("projects/soilgrids-isric/soc_mean").clip(roi) # soil organic carbon

# Function to select and average specific depth bands
def average_depths(image, name):
    depths = ['0-5cm', '5-15cm', '15-30cm']
    selected_bands = image.select([f'.*_{depth}_mean' for depth in depths])
    averaged = selected_bands.reduce(ee.Reducer.mean()).rename(name)
    return averaged

# Apply unit conversions and depth aggregation
bdod = average_depths(bdod, "bdod").divide(100)  # cg/cm³ to g/cm³
cec = average_depths(cec, "cec").divide(10)  # mmol(c)/kg to cmol/kg
clay = average_depths(clay, "clay").divide(10)        # g/kg to g/100g (%)
nitro = average_depths(nitro, "nitro").divide(100)    # cg/kg to g/kg
ocd = average_depths(ocd, "ocd").divide(10)    # hg/m³ to kg/m³
ocs = average_depths(ocs, "ocs").divide(10)           # hg/m² to kg/m²
phh2o = average_depths(phh2o, "phh2o").divide(10)     # pH x 10 to pH
sand = average_depths(sand, "sand").divide(10)        # g/kg to g/100g (%)
soc = average_depths(soc, "soc").divide(10)           # dg/kg to g/kg

# Combine all the soil properties into a single image
soil_properties = bdod.addBands([cec, clay, nitro, ocd, ocs, phh2o, sand, soc])

# Export the final image
export_image(soil_properties, "soilgrids", region = roi, scale = 250)


## TerraClim Data

<!-- # Terraclim
  https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_TERRACLIMATE
Bring temperature and precipitation and calculate seasonality -->

<!-- ## Seasonality index

Walsh and Lawler 1981 -->

In [ ]:
terraclim = (
    ee.ImageCollection("IDAHO_EPSCOR/TERRACLIMATE")
    .filterDate("1985-01-01", "2019-12-31")
    .map(lambda image: image.reduceResolution(ee.Reducer.median(), bestEffort = True, maxPixels = 1024)
    .reproject(crs = "EPSG:4326", scale = 10000))
)

# Function to calculate yearly metrics with scaling applied after filtering
def calculate_yearly_metrics(year):

    def scale_and_aggregate(var, reducer):
        data = terraclim.select(var).filter(ee.Filter.calendarRange(year, year, "year"))
        if var != "pr":
            data = data.map(lambda img: img.multiply(0.1))
        return data.reduce(reducer).toInt16().rename(f"{var}_{year}")

    # Define which variables are processed with sum or mean reducers
    sum_vars = ["srad", "soil", "pr"]
    mean_vars = ["vpd", "aet"]

    # Aggregate sum variables (radiation, soil, precipitation)
    processed_vars = {var: scale_and_aggregate(var, ee.Reducer.sum()) for var in sum_vars}
    
    # Aggregate mean temperature (average of max and min)
    maxtemp = scale_and_aggregate("tmmx", ee.Reducer.mean())
    mintemp = scale_and_aggregate("tmmn", ee.Reducer.mean())
    processed_vars["temp"] = maxtemp.addBands(mintemp).reduce(ee.Reducer.mean()).float().rename(f"temp_{year}")
    
    # Aggregate other mean variables (vpd, aet)
    processed_vars.update({var: scale_and_aggregate(var, ee.Reducer.mean()) for var in mean_vars})

    # Calculate Seasonality Index (SI)
    mean_prec = processed_vars["pr"].divide(12)
    deviations = terraclim.select("pr").filter(ee.Filter.calendarRange(year, year, "year")) \
        .map(lambda img: img.multiply(0.1))\
        .map(lambda month: month.subtract(mean_prec).abs())
    si_band = deviations.reduce(ee.Reducer.sum()).divide(processed_vars["pr"]).float().rename(f"si_{year}")

    return ee.Image.cat([*processed_vars.values(), si_band])

# Create a dictionary for variables with filtering and scaling applied after
vars = {var: terraclim.select(var) for var in ["tmmx", "tmmn", "srad", "vpd", "soil", "aet", "pr"]}

# Calculate yearly metrics and combine into a single image
yearly_metrics = ee.Image.cat([calculate_yearly_metrics(year) for year in config.range_1985_2019])

# Function to calculate the mean for a given band pattern
def calculate_mean(band_pattern, new_name):
    return yearly_metrics.select(band_pattern).reduce(ee.Reducer.mean()).rename(new_name)

# Calculate the mean across all years for the desired variables
mean_metrics = {name: calculate_mean(f".*{name}.*", f"mean_{name}") \
                for name in ["pr", "srad", "temp", "vpd", "soil", "aet", "si"]}

# Combine the mean layers into a single image
yearly_terraclim = ee.Image.cat([yearly_metrics, *mean_metrics.values()])

# Export the final image
export_image(yearly_terraclim, "yearly_terraclim", scale = 10000)


## ERA5 Data

<!-- ## Yearly CWD

Calculated as in Malhi et al 2009 and Aragão et al 2007, considering ET fixed as 100mm/month.

[equations](https://imgur.com/o4lVmM7) -->

In [36]:

vars = ['volumetric_soil_water_layer_2_min', 'surface_pressure', 'surface_net_solar_radiation_sum', 'temperature_2m']
var_names = ['sm', 'vpd', 'rad', 'temp']

ERA5_L = ee.Image()

for var, var_name in zip(vars, var_names):
    dataset = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR').select(var)

    yearly_values = ee.Image()
    for year in range(1985, 2020):
        # Filter images for the specific year
        images_in_year = dataset.filter(ee.Filter.eq('year', year))
        # Reduce the images to get the mean or total temperature for that year
        if var_name == 'rad':
            summarized = images_in_year.sum()
        else:
            summarized = images_in_year.mean()
        summarized = summarized.set('year', year).rename(f'{var_name}_{year}')
        yearly_values = yearly_values.addBands(summarized)
    
    yearly_values = yearly_values.slice(1)
    
    if var_name == 'sm':
        yearly_values = yearly_values.float()
        total_mean = yearly_values.reduce(ee.Reducer.mean()).float().rename(f'{var_name}_mean')
    if var_name == 'rad':
        yearly_values = yearly_values.divide(3.154e+7)
        total_mean = yearly_values.reduce(ee.Reducer.sum()).rename(f'{var_name}_sum')
    if var_name == 'temp':
        yearly_values = yearly_values.subtract(273.15)
        total_mean = yearly_values.reduce(ee.Reducer.mean()).rename(f'{var_name}_mean')
    else:
        total_mean = yearly_values.reduce(ee.Reducer.mean()).rename(f'{var_name}_mean')

    ERA5_L = ERA5_L.addBands([total_mean, yearly_values])

export_image(ERA5_L.slice(1), "ERA5_L", scale = 11132) # default by ERA5-Land